In [39]:
import numpy as np

class LikelihoodEvaluator():
    def __init__(self, Xpoints, numMixtures):
        assert(Xpoints.ndim==2)

        self.Xpoints = Xpoints
        self.numPoints, self.dim = Xpoints.shape
        self.numMixtures = numMixtures

    def loglikelihood(self, means, diagCovs, weights):
        raise NotImplementedError

    __call__ = loglikelihood

In [40]:
class SingleCoreLL(LikelihoodEvaluator):

    def __init__(self, Xpoints, numMixtures):
        print("Single Core Implementation Chosen")
        LikelihoodEvaluator.__init__(self, Xpoints, numMixtures)

    def __str__(self):
        return "Single Core Implementation"



    def loglikelihood(self, means, diagCovs, weights):
        numMixtures = self.numMixtures



        #update if need be

        assert(means.shape == (numMixtures, self.dim))
        assert(diagCovs.shape == (numMixtures, self.dim))
        assert(len(weights)== numMixtures)


        numMixtures = len(weights)

        ll = np.zeros(self.numPoints)

        constMulti = self.dim / 2.0 * np.log(2 * np.pi)

        CovDet = np.zeros(numMixtures)

        for i in range(numMixtures):
            CovDet[i] = 1.0 / np.sqrt(np.prod(diagCovs[i]))

        for i in range(self.numPoints):
            for mixes in range(numMixtures):
                multiVal = 1

                temp = np.dot((self.Xpoints[i] - means[mixes]) / diagCovs[mixes], (self. Xpoints[i] - means[mixes]))
                temp *= -0.5
                ll[i] += weights[mixes] * np.exp(temp) * CovDet[mixes]

            ll[i] = np.log(ll[i]) - constMulti

        return np.sum(ll)


In [87]:
from sklearn.mixture.gaussian_mixture import GaussianMixture, _compute_precision_cholesky


class ScikitLL(LikelihoodEvaluator):
    """
    Fastest Single Core Version so far!
    """

    def __init__(self, Xpoints, numMixtures):
        print("Scikits Learn Implementation Chosen")
        super().__init__(Xpoints, numMixtures)
        self.evaluator = GaussianMixture(numMixtures, 'diag')
        self.Xpoints = Xpoints
        self.evaluator.fit(Xpoints)

    def __str__(self):
        return "SciKit's learn implementation Implementation"

    def loglikelihood(self, means, diagCovs, weights):
        self.evaluator.weights_ = weights
        self.evaluator.covariances_ = diagCovs
        self.evaluator.means_ = means
        self.evaluator.precisions_cholesky_ = _compute_precision_cholesky(diagCovs, "diag")

        return self.numPoints * np.sum(self.evaluator.score(self.Xpoints))


In [88]:
N = 1000
d = 13
K = 8

testX = np.random.random((N, d))
testMu = np.random.random((K, d))
testSigma = np.ones((K, d))
testWeights = np.ones(K)/K

In [89]:
sc = SingleCoreLL(testX, K)
slearn = ScikitLL(testX, K)

Single Core Implementation Chosen
Scikits Learn Implementation Chosen


In [90]:
sc.loglikelihood(testMu, testSigma, testWeights)


-12956.342354467166

In [91]:
slearn.loglikelihood(testMu, testSigma, testWeights)

-12956.342354467166

In [22]:
def setup():
    import numpy as np
    a = np.random.random((100,1))
    sk = scikitLL(a, 4)
    sk.evaluator.fit(a)

    Sin = SingleCoreLL(a,4)

    print(np.sum(sk.evaluator.score(a)))

    print(Sin.loglikelihood(sk.evaluator.means_, sk.evaluator.covars_, sk.evaluator.weights_))

import timeit
setupStr = """
import numpy as np
from __main__ import scikitLL, SingleCoreLL
a = np.random.random((100,1))
sk = scikitLL(a, 4)
sk.evaluator.fit(a)
Sin = SingleCoreLL(a,4)
"""
actuSK = """
np.sum(sk.evaluator.score(a))
"""

actuSin = """
Sin.loglikelihood(sk.evaluator.means_, sk.evaluator.covars_, sk.evaluator.weights_)
"""

print(timeit.timeit(actuSin, setupStr, number = 1000))

Scikits Learn Implementation Chosen
Single Core Implementation Chosen


/home/varun/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/varun/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/varun/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
/home/varun/.local/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: Depreca

2.6111055000073975


In [23]:

class GPULL(LikelihoodEvaluator):

    def __str__(self):
        return "GPU Implementation"

    def __init__(self, Xpoints, numMixtures):
        print("GPU Implementation Chosen")
        LikelihoodEvaluator.__init__(self, Xpoints, numMixtures)

        #Pycuda imports
        import pycuda.autoinit
        from pycuda import gpuarray
        from pycuda.compiler import SourceModule

        self.gpuarray = gpuarray

        with open("KernelV2.cu") as f:

            if self.numPoints >= 1024:
                mod = SourceModule(f.read().replace('512', '1024'))
                self.numThreads = 1024
            else:
                mod = SourceModule(f.read())
                self.numThreads = 512

        if self.numPoints > self.numThreads:
            self.numBlocks = self.numPoints / self.numThreads
            if self.numPoints % self.numThreads != 0: self.numBlocks += 1
        else:
            self.numBlocks = 1

        print "numBlocks: {}, numPoints: {}".format(self.numBlocks, self.numPoints)
        #Set the right number of threads and blocks given the datasize
        #Using a max of 1024 threads, fix correct blocksize


        self.likelihoodKernel = mod.get_function("likelihoodKernel")
        self.likelihoodKernel.prepare('PPPPiiiP')

        self.Xpoints = self.Xpoints.astype(np.float32)
        self.Xpoints = gpuarray.to_gpu_async(self.Xpoints)

        self.means_gpu = gpuarray.zeros(shape = (self.numMixtures, self.dim), dtype = np.float32)
        self.diagCovs_gpu = gpuarray.zeros(shape = (self.numMixtures, self.dim), dtype = np.float32)
        self.weights_gpu = gpuarray.zeros(shape = self.numMixtures, dtype = np.float32)

        self.llVal = gpuarray.zeros(shape = self.numBlocks,  dtype=np.float32)

        #Allocate Memory for all our computations


    def loglikelihood(self, means, diagCovs, weights):


        assert(means.shape == (self.numMixtures, self.dim))
        assert(diagCovs.shape == (self.numMixtures, self.dim))
        assert(len(weights)== self.numMixtures)

        if means.dtype != np.float32:
            means = means.astype(np.float32)
        if diagCovs.dtype != np.float32:
            diagCovs = diagCovs.astype(np.float32)
        if weights.dtype != np.float32:
            weights = weights.astype(np.float32)


        #quick sanity checks
        self.means_gpu.set_async(means)
        self.diagCovs_gpu.set_async(diagCovs)
        self.weights_gpu.set(weights)

        self.likelihoodKernel.prepared_call((self.numBlocks, 1), (self.numThreads, 1, 1),
                                       self.Xpoints.gpudata, self.means_gpu.gpudata, self.diagCovs_gpu.gpudata,
                                       self.weights_gpu.gpudata,
                                       self.dim, self.numPoints, self.numMixtures,
                                       self.llVal.gpudata)

        ll = self.gpuarray.sum(self.llVal).get()
        return ll

SyntaxError: invalid syntax (<ipython-input-23-349609f2285d>, line 33)

In [24]:
from sklearn.mixture import GaussianMixture as GMMEval


In [26]:
skmod = GMMEval(8, 'diag')

In [30]:
skmod.fit(testX)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=8, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [32]:
skmod.covariances_.shape

(8, 13)

In [35]:
skmod.score(testX)

-0.6500099582999345